# Files and Regular Expressions

## Reading files

In [1]:
import scala.io.Source

import scala.io.Source

In [6]:
val file = Source.fromFile("test.csv")

file: io.BufferedSource = non-empty iterator

In [7]:
val lines = file.getLines
lines.length

lines: Iterator[String] = empty iterator
res6_1: Int = 419

In [8]:
file.close()

* Iterating over `Source` object directly iterates through one
character at a time.
* `Source.buffered` returns Iterator that has the capability to
peek without moving the iterator.
* Small files contents to string using `Source.mkString`

* `Source.fromURL`, `Source.fromString`, `Source.stdin` can be used to
read inputs from various sources.

* Scala's io is very limited. Looks like most often we need
to use java standard packages like `java.io`, `java.nio`

* For reading binary files, we have to use java class `FileInputStream`

* For writing files, we have to use `java.io.PrintWriter`

## Serialization

* Extend `scala.Serializable` trait
* Scala collections are serializable.
* To serialize and deserialize we use `ObjectOutputStream`
and `ObjectInputStream` respectively (binary serialization).


## Process control

* `scala.sys.process` package provides shell utilities.

In [9]:
import scala.sys.process._

// below shell command strings are converted to ProcessBuilder objects
// process package uses scala's implicit feature
// ! - returns the exit code of the the program
// 0 - success and non zero - failure
"echo 'Hello'".!

Hello


import scala.sys.process._

// below shell command strings are converted to ProcessBuilder objects
// process package uses scala's implicit feature
// ! - returns the exit code of the the program
// 0 - success and non zero - failure

res8_1: Int = 0

In [10]:
// to get the output as string use !!
val out = "echo 'hello'".!!
println(out)

hello



out: String = """hello
"""

In [11]:
// since |, >>, > all are known to be used as operators(bitwise, relational)
// we prefix those with # when we need to use them in the shell context
//pipe
"echo 'Hello'".#|("cat").!

// redirect to file
"echo 'Hello'".#>(new java.io.File("output.txt")).!

// append to file
"echo 'World'".#>>(new java.io.File("output.txt")).!

// redirect to stdin
// append to file
"cat".#<(new java.io.File("output.txt")).!

// combine shell command execution using &&
"echo 'Hello'".#&&("echo 'world'").!

Hello
Hello
World
Hello
world


res10_0: Int = 0
res10_1: Int = 0
res10_2: Int = 0
res10_3: Int = 0
res10_4: Int = 0

In [20]:
// Using Process object directly
val envs = List[(String, String)](
    ("MY_ENV", "123456789")
)
val p = Process("""
echo "hello"
""", cwd=None, envs:_*)
p.!

hello


envs: List[(String, String)] = List(("MY_ENV", "123456789"))
p: ProcessBuilder = [echo, hello]
res19_2: Int = 0

## Regular Expressions

* `scala.util.matching.Regex` is the regex class
* `"some_regex_pattern".r` returns regex object.
* Prefer using **raw** strings(triple quoted) for regex with backslashes.
* `findAllIn`, `findFirstIn`, `replaceFirstIn`, `replaceAllIn` are
some of the useful methods.

In [22]:
val content = "1 apple, 20 banana, 10 watermelons"

// find quantities
val regexPattern = """[0-9]+""".r

for (m <- regexPattern.findAllIn(content))
    println(m)

val firstMatch: Option[String] = regexPattern.findFirstIn(content)
println(firstMatch)

1
20
10
Some(1)


content: String = "1 apple, 20 banana, 10 watermelons"
regexPattern: scala.util.matching.Regex = [0-9]+
firstMatch: Option[String] = Some("1")

In [23]:
// Suppose we need to mask all the quantities with XX
println(regexPattern.replaceAllIn(content, "XX"))

// suppose we need to double the quantities of every fruit
val newContent = regexPattern.replaceSomeIn(content,
    m => Some(s"${m.matched.toInt * 2}"))

println(newContent)

XX apple, XX banana, XX watermelons
2 apple, 40 banana, 20 watermelons


newContent: String = "2 apple, 40 banana, 20 watermelons"

## Regex groups

In [24]:
val pattern = """([0-9]+)\s+([a-zA-Z]+)""".r

for (matchedText <- pattern.findAllIn(content))
    println(matchedText)

1 apple
20 banana
10 watermelons


pattern: scala.util.matching.Regex = ([0-9]+)\s+([a-zA-Z]+)

In [26]:
// to fetch the matched groups we need to use findFirstMatchIn
// and findAllMatchIn methods
for (m <- pattern.findAllMatchIn(content)) {
    println(m.matched) // entire match
    println(m.group(0), m.start(0), m.end(0)) // zeroth group is entire match
    println(m.group(1), m.start(1), m.end(1)) // first group
    println(m.group(2), m.start(2), m.end(2)) // second group
}

1 apple
(1 apple,0,7)
(1,0,1)
(apple,2,7)
20 banana
(20 banana,9,18)
(20,9,11)
(banana,12,18)
10 watermelons
(10 watermelons,20,34)
(10,20,22)
(watermelons,23,34)
